In [1]:
import ccxt
import numpy as np
import pandas as pd

import plotly.graph_objects as go

ex = ccxt.binance()

In [2]:
def get_ohlcv(symbol, since, timeframe, limit):
    from_ts = ex.parse8601(since)

    ts = []
    time = []
    open = []
    high = []
    low = []
    close = []
    volume = []

    mid = []
    open_close_dif = []
    low_high_dif = []
    open_close_dif_abs = []
    low_high_dif_abs = []

    ohlcvs = ex.fetch_ohlcv(symbol, timeframe, since=from_ts, limit=limit)

    for ohlcv in ohlcvs:
        t, o, h, l, c, v = ohlcv
        ts.append(t)
        time.append(pd.to_datetime(t, unit='ms'))
        open.append(o)
        high.append(h)
        low.append(l)
        close.append(c)
        volume.append(v)

        mid.append((h+l) / 2)
        open_close_dif.append(o-c)
        low_high_dif.append(l-h)
        open_close_dif_abs.append(abs(o-c))
        low_high_dif_abs.append(abs(l-h))

    return pd.DataFrame({"ts": ts, "time": time, "open": open, "high": high, "low": low, "close": close, "volume": volume, "mid": mid, "open_close_dif": open_close_dif, "low_high_dif": low_high_dif, "open_close_dif_abs": open_close_dif_abs, "low_high_dif_abs": low_high_dif_abs})

In [3]:
def fill_ohcls(ohcls):
    diff = np.abs(np.diff(ohcls['mid']))

    mid_shifted = ohcls['mid'][1:].to_numpy()
    mid_without_last = ohcls['mid'][:-1].to_numpy()

    div = mid_shifted / mid_without_last
    log_diff = np.log(div)

    ohcls.loc[1:, ['diff']] = diff
    ohcls.loc[1:, ['log_diff']] = np.abs(log_diff)

    return ohcls[1:]

In [4]:
def plot_volatility(ohcls):
    volatility = go.Scatter(x=ohcls['time'], y=ohcls["log_diff"], name = 'volatility', mode='lines')

    time_period = ohcls.iloc[[0,-1]]['time']
    mean = ohcls['log_diff'].mean()
    y_sigma1 = ohcls['log_diff'].std()
    y_sigma2 = y_sigma1 * 2
    y_sigma3 = y_sigma1 * 3

    mean = go.Scatter(x=time_period, y = [mean, mean], name = 'mean', mode='lines', line={'width': 1})
    sigma1 = go.Scatter(x=time_period, y = [y_sigma1, y_sigma1], name = 'sigma1', mode='lines', line={'width': 1})
    sigma2 = go.Scatter(x=time_period, y = [y_sigma2, y_sigma2], name = 'sigma2', mode='lines', line={'width': 1})
    sigma3 = go.Scatter(x=time_period, y = [y_sigma3, y_sigma3], name = 'sigma3', mode='lines', line={'width': 1})

    fig = go.Figure()
    fig.add_trace(volatility)
    fig.add_trace(mean)
    fig.add_trace(sigma1)
    fig.add_trace(sigma2)
    fig.add_trace(sigma3)
    fig.show()

In [5]:
def start(symbol, since, timeframe, limit = 1000):
    """
    start volatility investigation
    :param str symbol: Unified CCXT market symbol
    :param str timeframe: "5m","15m","30m","1h","2h","4h","6h","12h", or "1d"
    :param str since: date format: '2022-10-03 00:00:00'
    :param int|None limit: default 1000
    """
    ohcls = get_ohlcv(symbol, since, timeframe, limit)
    ohcls = fill_ohcls(ohcls)
    plot_volatility(ohcls)


In [8]:
start('BTC/USDT', '2022-10-01 00:00:00', '15m', 5000)

In [10]:
start('BTC/USDT', '2022-10-01 00:00:00', '5m', 5000)